In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder 

In [2]:
df_codi = pd.read_csv('pairs_limpieza.csv', index_col=0)
df_codi.head(2)

,case_number,year,type,country,area,location,activity,name,age,species,injury,fecha_limpia,fatal_limpio,sexo,type_sin_nulos
0,1800.00.00,1800,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,NaN,NaN,"FATAL, all onboard were killed by sharks",unspecified,Y,F,Unprovoked
1,1797.05.28.R,1797,Unprovoked,unknown,NaN,NaN,Dropped overboard,child,NaN,NaN,FATAL,May,Y,M,Unprovoked


# Ejercicio 1
Lo primero que debemos hacer es identificar cuáles de nuestras columnas es categóricas. Extraed el nombre de las columnas que contienen variables categóricas.

In [3]:
df_codi.dtypes

case_number       object
year               int64
type              object
country           object
area              object
location          object
activity          object
name              object
age               object
species           object
injury            object
fecha_limpia      object
fatal_limpio      object
sexo              object
type_sin_nulos    object
dtype: object

In [4]:
col_categorica  = df_codi.select_dtypes(include = 'object')

In [5]:
col_categorica.columns

Index(['case_number', 'type', 'country', 'area', 'location', 'activity',
       'name', 'age', 'species', 'injury', 'fecha_limpia', 'fatal_limpio',
       'sexo', 'type_sin_nulos'],
      dtype='object')

# Ejercicio 2
Si nos fijamos en la columna de country es muy complicado codificar esa columna ya que hay muchísimas categorías. Para ello, vamos a usar una librería que llamada geopy que nos devuelve las coordenadas de un pais, ciudad, pueblo. El objetivo de esto es poder clasificar nuestros paises en Hemisferio Norte y Sur para poder hacer una codificación más eficaz. Os dejamos por aquí la documentación de la librería para que veais como funciona y el código:

In [5]:
def extraer_coordenadas(x):
    try:
        geolocator = Nominatim(user_agent = "ana")
        location = geolocator.geocode(x)
        return location.lat, location.long
    except:
        return "no hay datos"

In [6]:
df_codi = pd.read_csv('attacks_limpiando_con_coordenadas.csv', index_col=0)
df_codi.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.6585116,-102.0077097


In [7]:
df_codi["species_"] = df_codi["species_"].rename("species")

In [8]:
df_codi.dtypes

year              int64
type             object
country          object
age             float64
species_         object
fecha_limpia     object
fatal            object
sex              object
latitud          object
longitud         object
dtype: object

In [9]:
df_categorica  = df_codi.select_dtypes(include = 'object')

In [10]:
df_categorica.columns

Index(['type', 'country', 'species_', 'fecha_limpia', 'fatal', 'sex',
       'latitud', 'longitud'],
      dtype='object')

- Como ya tenemos las coordenadas, deberéis codificar la columna en función de si pertenecen al Hemisferio Norte (HN)(postivo) o Sur (HS)(negativo). Aquellas que pertencezcan al HN le pondremos un 1 y a las del HS le pondremos un 0.

In [11]:
df_categorica_copia = df_categorica.copy()

In [12]:
latitud_nueva = []

for i in df_categorica_copia['latitud']:
    if i < '0':
        latitud_nueva.append('HS')
    else: 
        latitud_nueva.append('HN')

In [12]:
latitud_nueva

NameError: name 'latitud_nueva' is not defined

In [14]:
df_latitud = pd.DataFrame(latitud_nueva)

,0
0,HN
1,HN
2,HN


In [16]:
df_latitud.value_counts()

HN    3495
HS    2723
dtype: int64

In [17]:
df_categorica_copia['latitud'] = df_latitud

Usamos el método ***map*** para realizar la codificación en nuestra columna

In [18]:
mapa = {'HN': 1, 'HS': 0}

In [19]:
df_categorica_copia["latitud"] = df_categorica_copia["latitud"].map(mapa)

Comprobamos que se han realizados el mapeo y pasamos a reemplazarlo en la columna del df original

In [20]:
df_categorica_copia["latitud"].value_counts()

1    3495
0    2723
Name: latitud, dtype: int64

In [21]:
df_codi['latitud'] = df_categorica_copia['latitud']

# Ejercicio 3
Aplicad un get_dummies a las columnas de fatal y age.

In [22]:
dummies_fatal = pd.get_dummies(df_categorica["fatal"], prefix_sep = "_", prefix = "fatal", dtype = int)
dummies_fatal.head(2)

,fatal_N,fatal_Unknown,fatal_Y
0,1,0,0
1,1,0,0


In [23]:
df_codi = pd.concat([df_codi, dummies_fatal], axis = 1)
df_codi.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,fatal_N,fatal_Unknown,fatal_Y
0,2018,Boating,usa,57.0,White shark,Jun,N,F,1,-100.445882,1,0,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,1,-100.445882,1,0,0


Como la columna age es de tipo float, primero tendremos que convertirlo a tipo string. Usaremos el método cut para dividir las edades en rangos. A partir de ahí, ya podremos codificarlo.

In [24]:
df_codi['age'].describe()

count    6218.000000
mean       27.163397
std        10.362343
min         1.000000
25%        22.000000
50%        27.000000
75%        27.000000
max        87.000000
Name: age, dtype: float64

In [25]:
bins_age = pd.cut(df_codi['age'], bins = 3, labels=['jovenes', 'adultos', 'ancianos'])

In [26]:
dummies_age = pd.get_dummies(bins_age, prefix_sep = "_", prefix = "age", dtype = int)
dummies_age.head(2)

,age_jovenes,age_adultos,age_ancianos
0,0,1,0
1,1,0,0


In [27]:
df_codi = pd.concat([df_codi, dummies_age], axis = 1)
df_codi.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,fatal_N,fatal_Unknown,fatal_Y,age_jovenes,age_adultos,age_ancianos
0,2018,Boating,usa,57.0,White shark,Jun,N,F,1,-100.445882,1,0,0,0,1,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,1,-100.445882,1,0,0,1,0,0


# Ejercicio 4
Aplicad un Label Encoding a la columna species , fecha y type.

In [28]:
le = LabelEncoder()

In [30]:
df_col_elegidas = df_categorica[['type','species_','fecha_limpia']]

KeyError: "['species'] not in index"

In [83]:

for col in df_col_elegidas:
    df_col_elegidas[col]=le.fit_transform(df_col_elegidas[col])

/tmp/ipykernel_15705/3797466698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_col_elegidas[col]=le.fit_transform(df_col_elegidas[col])


In [84]:
df_col_elegidas

,type,species_,fecha_limpia
0,1,4,7
1,7,3,7
2,3,3,7
3,7,3,7
4,4,2,7
...,...,...,...
6213,7,2,14
6214,7,2,14
6215,7,2,14
6216,7,3,14


In [85]:
df_codi = pd.concat([df_codi, df_col_elegidas], axis = 1)
df_codi.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,fatal_N,fatal_Unknown,fatal_Y,age_jovenes,age_adultos,age_ancianos,type,species_,fecha_limpia
0,2018,Boating,usa,57.0,White shark,Jun,N,F,1,-100.445882,1,0,0,0,1,0,1,4,7
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,1,-100.445882,1,0,0,1,0,0,7,3,7


BORRAR COLUMNAS ORIGINALES USANDO EL INDICE (ILOC)

GUARDAR CSV

In [86]:
df_codi.to_csv('pairs_limpieza.csv')

In [89]:
df_codi['age'] = df_codi['age'].astype('int64', copy = True, errors = 'raise')

In [90]:
df_codi.dtypes

year              int64
type             object
country          object
age               int64
species_         object
fecha_limpia     object
fatal            object
sex              object
latitud           int64
longitud         object
fatal_N           int64
fatal_Unknown     int64
fatal_Y           int64
age_jovenes       int64
age_adultos       int64
age_ancianos      int64
type              int64
species_          int64
fecha_limpia      int64
dtype: object